In [48]:
# 초기 설정
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 100% !important; }</style>"))
pd.set_option("display.max_columns", 40)
import missingno as msno
%matplotlib inline

import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

##### 하단 정제 데이터 부분에서 모델 돌리세용 ######

##### 변수 추가

In [ ]:
#BABIP
sdata['BABIP'] = (sdata['안타'] - sdata['홈런'])/(sdata['타수'] -sdata['삼진']-sdata['홈런'] -sdata['희비'])

In [ ]:
#외국인
import re
names = sdata.이름.unique()
sdata['외국인'] = None
for name in names:
    if re.findall('[에, 브, 워, 대, 피, 히, 버, 러, 칸, 루, 필, 파, 스, 아,마, 가, 초, 모, 로, 발, 번, 테, 호]', name[0]):
        sdata['외국인'][sdata['이름'] == name] = 1
    else:
        sdata['외국인'][sdata['이름'] == name] = 0
        
        
a = ['나바로', '조쉬벨', '고메즈']
for name in a:
    sdata['외국인'][sdata['이름'] == name] = 1

In [ ]:
# 나이 컬럼화
sdata['나이C'] = sdata.나이.apply(lambda x : 0 if x <= 23 else x)
sdata['나이C'] = sdata.나이C.apply(lambda x : 1 if 26 >= x > 23 else x)
sdata['나이C'] = sdata.나이C.apply(lambda x : 2 if 33 >= x >26 else x)
sdata['나이C'] = sdata.나이C.apply(lambda x : 3 if 37 >= x >33 else x)
sdata['나이C']= sdata.나이C.apply(lambda x : 4 if 40 >= x > 37 else x) 
sdata['나이C']= sdata.나이C.apply(lambda x : 5 if x >= 40 else x) 

In [ ]:
# 이적
sdata['이적'] = None
for x in df['이름']:
    sdata['이적'][sdata['이름'] == x] = len(sdata[sdata['이름'] == x]['팀'].unique())
    

In [ ]:
df['이적C'] = df.이적.apply(lambda x: 0 if x == 1 else x)
df['이적C'] = df.이적.apply(lambda x: 1 if x == 2 else x)
df['이적C'] = df.이적.apply(lambda x: 2 if x == 3 else x)
df['이적C'] = df.이적.apply(lambda x: 3 if x >= 3 else x)

In [2]:
sdata.columns

Index(['ID', '이름', '시즌', '팀', '포지션', '나이', 'G', '타석', '타수', '득점', '안타', '타1',
       '타2', '타3', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살',
       '희타', '희비', '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC', 'twoBLUCK',
       'threeBLUCK', 'ISO', 'BBK', 'ISOD', '연봉', '횟수', '경험'],
      dtype='object')

#### 정제 데이터 넣고 돌리는 부분

In [61]:
test = pd.read_csv('test_datas0402.csv')
train = pd.read_csv('train_datas0402.csv')

In [50]:
#sdata

,ID,이름,시즌,팀,포지션,나이,G,타석,타수,득점,안타,타1,타2,타3,홈런,루타,타점,도루,도실,볼넷,...,타율,출루,장타,OPS,wOBA,wRC,twoBLUCK,threeBLUCK,ISO,BBK,ISOD,연봉,횟수,경험,BABIP,로그연봉,외국인,나이C,이적,이적C
0,1,이대호,2019,롯데,DH,38,135.0,549.0,485.0,48.0,138.0,98.0,23.0,1.0,16.0,211.0,88.0,0.0,0.0,48.0,...,0.285,0.355,0.435,0.790,0.360,118.3,0.056931,0.002475,0.150,0.030965,0.070,250000.0,1,0,0.307305,12.429216,0,4,1,0
1,3,박석민,2019,NC,3B,35,112.0,444.0,360.0,49.0,96.0,59.0,17.0,1.0,19.0,172.0,74.0,1.0,2.0,65.0,...,0.267,0.393,0.478,0.871,0.393,139.3,0.062500,0.003676,0.211,0.009009,0.126,75000.0,1,0,0.286245,11.225243,0,3,1,0
2,4,박병호,2019,키움,1B,34,122.0,532.0,432.0,92.0,121.0,66.0,22.0,0.0,33.0,242.0,98.0,0.0,1.0,78.0,...,0.280,0.399,0.560,0.959,0.422,165.8,0.078014,0.000000,0.280,0.073308,0.119,150000.0,1,0,0.322344,11.918391,0,3,1,0
3,5,최형우,2019,KIA,DH,37,136.0,555.0,456.0,65.0,137.0,88.0,31.0,1.0,17.0,221.0,86.0,0.0,1.0,85.0,...,0.300,0.413,0.485,0.897,0.406,150.3,0.085635,0.002762,0.185,-0.014414,0.113,150000.0,1,0,0.338028,11.918391,0,3,1,0
4,6,서건창,2019,키움,2B,31,113.0,486.0,426.0,67.0,128.0,100.0,23.0,3.0,2.0,163.0,41.0,17.0,3.0,49.0,...,0.300,0.373,0.383,0.755,0.352,112.6,0.061497,0.008021,0.083,0.002058,0.073,35000.0,1,0,0.341463,10.463103,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,206,문규현,2019,롯데,3B,37,40.0,72.0,64.0,7.0,16.0,14.0,2.0,0.0,0.0,18.0,10.0,0.0,0.0,3.0,...,0.250,0.271,0.281,0.553,0.250,43.9,0.037037,0.000000,0.031,0.097222,0.021,18000.0,1,0,0.313725,9.798127,0,3,1,0
78,232,김민식,2019,KIA,C,31,53.0,116.0,96.0,6.0,16.0,14.0,2.0,0.0,0.0,18.0,14.0,2.0,0.0,15.0,...,0.167,0.293,0.188,0.481,0.249,42.3,0.029851,0.000000,0.021,0.120690,0.126,14000.0,1,0,0.246154,9.546813,0,2,1,0
79,247,류지혁,2019,두산,SS,26,118.0,316.0,276.0,36.0,69.0,61.0,6.0,2.0,0.0,79.0,34.0,18.0,5.0,24.0,...,0.250,0.323,0.286,0.609,0.289,69.9,0.025424,0.008475,0.036,0.050633,0.073,12500.0,1,0,0.296137,9.433484,0,1,1,0
80,255,정진호,2019,두산,RF,32,66.0,110.0,96.0,13.0,20.0,19.0,0.0,0.0,1.0,23.0,3.0,5.0,2.0,11.0,...,0.208,0.290,0.240,0.529,0.254,47.1,0.000000,0.000000,0.032,0.118182,0.082,11300.0,1,0,0.267606,9.332558,0,2,1,0


In [59]:
#train = sdata[(sdata['시즌'] <= 2018)]
#test = sdata[(sdata['시즌'] == 2019)]

In [62]:
train

,ID,이름,시즌,팀,포지션,나이,G,타석,타수,득점,안타,타1,타2,타3,홈런,루타,타점,도루,도실,볼넷,...,타율,출루,장타,OPS,wOBA,wRC,twoBLUCK,threeBLUCK,ISO,BBK,ISOD,연봉,횟수,경험,BABIP,로그연봉,외국인,나이C,이적,이적C
0,0,테임즈,2014,NC,1B,29,125.0,514.0,443.0,95.0,152.0,79.0,30.0,6.0,37.0,305.0,121.0,11.0,2.0,58.0,...,0.343,0.422,0.688,1.111,0.464,168.3,0.097720,0.019544,0.345,0.079767,0.079,25000.0,1,0,0.382060,10.126631,1,2,1,0
1,0,테임즈,2015,NC,1B,30,142.0,595.0,472.0,130.0,180.0,86.0,42.0,5.0,47.0,373.0,140.0,40.0,8.0,103.0,...,0.381,0.498,0.790,1.288,0.530,222.3,0.125749,0.014970,0.409,-0.020168,0.117,125000.0,2,1,0.406728,11.736069,1,2,1,0
2,0,테임즈,2016,NC,1B,31,123.0,529.0,436.0,118.0,140.0,67.0,30.0,3.0,40.0,296.0,121.0,13.0,4.0,74.0,...,0.321,0.427,0.679,1.106,0.463,170.5,0.102389,0.010239,0.358,0.054820,0.106,125000.0,3,2,0.349650,11.736069,1,2,1,0
3,1,이대호,2010,롯데,3B,29,127.0,552.0,478.0,99.0,174.0,117.0,13.0,0.0,44.0,319.0,133.0,0.0,2.0,61.0,...,0.364,0.444,0.667,1.111,0.482,192.4,0.036415,0.000000,0.303,0.028986,0.080,39000.0,1,0,0.367232,10.571317,0,2,1,0
4,1,이대호,2011,롯데,1B,30,133.0,570.0,493.0,76.0,176.0,122.0,26.0,1.0,27.0,285.0,113.0,2.0,0.0,63.0,...,0.357,0.433,0.578,1.011,0.455,180.0,0.064039,0.002463,0.221,-0.005263,0.076,63000.0,2,1,0.372500,11.050890,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,379,박기혁,2010,롯데,SS,30,48.0,166.0,153.0,10.0,33.0,30.0,3.0,0.0,0.0,36.0,13.0,2.0,0.0,6.0,...,0.216,0.248,0.235,0.484,0.225,22.8,0.022222,0.000000,0.019,0.072289,0.032,12000.0,1,0,0.246269,9.392662,0,2,1,0
481,885,워스,2017,SK,DH,33,3.0,11.0,9.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,...,0.111,0.273,0.111,0.384,0.217,12.4,0.000000,0.000000,0.000,0.000000,0.162,70000.0,1,0,0.142857,11.156251,1,2,1,0
482,941,루츠,2015,두산,3B,30,8.0,30.0,27.0,2.0,3.0,2.0,0.0,0.0,1.0,6.0,3.0,0.0,0.0,2.0,...,0.111,0.167,0.222,0.389,0.175,-8.2,0.000000,0.000000,0.111,0.133333,0.056,55000.0,1,0,0.105263,10.915088,1,2,1,0
483,950,모넬,2017,kt,1B,32,28.0,105.0,85.0,10.0,14.0,8.0,4.0,0.0,2.0,24.0,9.0,2.0,1.0,17.0,...,0.165,0.305,0.282,0.587,0.276,50.2,0.072727,0.000000,0.117,0.104762,0.140,45000.0,1,0,0.226415,10.714418,1,2,1,0


In [63]:
import statsmodels.api as sm
model = sm.OLS.from_formula('연봉 ~  np.log(나이) +  scale(np.log(G)) + scale(타석) + np.log(scale(안타)) +  (np.log(볼넷)) + \
                            (np.log(사구)) + scale(고4) + scale(np.log(삼진)) + np.log(병살) +\
                            np.log(scale(희타)) + scale(희비)  + np.log((twoBLUCK)) + np.log(scale(threeBLUCK)) + (scale(ISO)) +\
                            np.log(scale(ISOD)) ' , data=train)
# model = sm.OLS.from_formula('연봉 ~ ' + '+'.join(s_col) , data=train)
result = model.fit()
#print(result.summary())

In [24]:
train.columns

Index(['ID', '이름', '시즌', '팀', '포지션', '나이', 'G', '타석', '타수', '득점', '안타', '타1',
       '타2', '타3', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살',
       '희타', '희비', '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC', 'twoBLUCK',
       'threeBLUCK', 'ISO', 'BBK', 'ISOD', '연봉', '횟수', '경험', 'BABIP', '로그연봉',
       '외국인', '나이C', '이적', '이적C'],
      dtype='object')

In [25]:

train_x = train[['시즌', '팀', '포지션', '나이', 'G', '타석', '타수', '득점', '안타', '타1',
       '타2', '타3', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살',
       '희타', '희비', '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC', 'twoBLUCK',
       'threeBLUCK', 'ISO', 'BBK', 'ISOD', '횟수', '경험', 'BABIP', 
       '외국인', '나이C', '이적', '이적C']]
train_y = train[['연봉']]
train_y_log = train[['로그연봉']]

test_x = test[['시즌', '팀', '포지션', '나이', 'G', '타석', '타수', '득점', '안타', '타1',
       '타2', '타3', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살',
       '희타', '희비', '타율', '출루', '장타', 'OPS', 'wOBA', 'wRC', 'twoBLUCK',
       'threeBLUCK', 'ISO', 'BBK', 'ISOD', '횟수', '경험', 'BABIP', 
       '외국인', '나이C', '이적', '이적C']]
test_y = test[['연봉']]
test_y_log = test['로그연봉']

In [64]:
#col2 = ['시즌', '나이',  'G', '타석',   '도실', '볼넷',\
#        '사구', '고4', '삼진', '병살', '희타', '희비', '타율',   'WAR', 'twoBLUCK',\
#        'threeBLUCK', 'ISO',  'ISOD']
#s_col = [f'scale({c})' for c in col2]
model = sm.OLS.from_formula('연봉 ~  G' , data=train)
# model = sm.OLS.from_formula('연봉 ~ ' + '+'.join(s_col) , data=train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                     연봉   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     18.94
Date:                Thu, 02 Apr 2020   Prob (F-statistic):           1.65e-05
Time:                        21:07:15   Log-Likelihood:                -5723.5
No. Observations:                 485   AIC:                         1.145e+04
Df Residuals:                     483   BIC:                         1.146e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.317e+04   4989.912      4.644      0.0

##### 1. 전체 모형

In [69]:
model = sm.OLS.from_formula('연봉 ~  나이+ G+ 타석+ 타수+ 득점+ 안타+ 타1+ 타2+ 타3+\
                                홈런+ 루타+ 타점+ 도루+ 도실+ 볼넷+ 사구+ 고4+ 삼진+ 병살+  희타+ 희비+ \
                                타율+ 출루+ 장타+ OPS+ wOBA+ wRC+ twoBLUCK+ threeBLUCK+ ISO+ BBK+ ISOD+ 횟수+\
                                경험 + BABIP+외국인+ 나이C+ 이적+ 이적C', data=train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                     연봉   R-squared:                       0.547
Model:                            OLS   Adj. R-squared:                  0.513
Method:                 Least Squares   F-statistic:                     15.99
Date:                Thu, 02 Apr 2020   Prob (F-statistic):           1.02e-57
Time:                        21:12:14   Log-Likelihood:                -5540.8
No. Observations:                 485   AIC:                         1.115e+04
Df Residuals:                     450   BIC:                         1.130e+04
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -6.651e+04   1.74e+04     -3.825      0.0

In [68]:
df = train
from sklearn.model_selection import KFold

scores = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]
    
    model = sm.OLS.from_formula('연봉 ~  나이+ G+ 타석+ 타수+ 득점+ 안타+ 타1+ 타2+ 타3+\
                                홈런+ 루타+ 타점+ 도루+ 도실+ 볼넷+ 사구+ 고4+ 삼진+ 병살+  희타+ 희비+ \
                                타율+ 출루+ 장타+ OPS+ wOBA+ wRC+ twoBLUCK+ threeBLUCK+ ISO+ BBK+ ISOD+ 횟수+\
                                경험 + BABIP+외국인+ 나이C+ 이적+ 이적C' , data=df_train)
    result = model.fit()
    
    pred = result.predict(df_test)
    rss = ((df_test.연봉 - pred) ** 2).sum()
    tss = ((df_test.연봉 - df_test.연봉.mean())** 2).sum()
    rsquared = 1 - rss / tss
    
    scores[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result.rsquared, rsquared))

학습 R2 = 0.55592879, 검증 R2 = 0.45577089
학습 R2 = 0.55656713, 검증 R2 = 0.46154012
학습 R2 = 0.53599672, 검증 R2 = 0.53552159
학습 R2 = 0.56871605, 검증 R2 = 0.35868853
학습 R2 = 0.56080194, 검증 R2 = 0.43331498


#### 2. 모델2

In [77]:
model = sm.OLS.from_formula('로그연봉 ~ C(시즌) + 포지션 + scale(G) + C(팀) + scale(홈런) + scale(타3)\
                            + scale(타석) + scale(득점) + scale(사구) + scale(고4) + scale(도루)\
                            + scale(삼진) + scale(병살) + scale(희타) + scale(희비) + scale(타율) + 외국인\
                            + scale(OPS) + C(나이C):횟수', data=train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     16.67
Date:                Thu, 02 Apr 2020   Prob (F-statistic):           8.64e-72
Time:                        21:35:21   Log-Likelihood:                -243.89
No. Observations:                 485   AIC:                             585.8
Df Residuals:                     436   BIC:                             790.8
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         9.5062      0.218     43.608

In [78]:
df = train
from sklearn.model_selection import KFold

scores = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]
    
    model = sm.OLS.from_formula('로그연봉 ~ C(시즌) + 포지션 + scale(G) + C(팀) + scale(홈런) + scale(타3)\
                            + scale(타석) + scale(득점) + scale(사구) + scale(고4) + scale(도루)\
                            + scale(삼진) + scale(병살) + scale(희타) + scale(희비) + scale(타율) + 외국인\
                            + scale(OPS) + C(나이C):횟수' , data=df_train)
    result = model.fit()
    
    pred = result.predict(df_test)
    rss = ((df_test.로그연봉 - pred) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean())** 2).sum()
    rsquared = 1 - rss / tss
    
    scores[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result.rsquared, rsquared))

학습 R2 = 0.65949481, 검증 R2 = 0.53668681
학습 R2 = 0.66369142, 검증 R2 = 0.46238439
학습 R2 = 0.65373213, 검증 R2 = 0.55063842
학습 R2 = 0.65665239, 검증 R2 = 0.53377046
학습 R2 = 0.65596735, 검증 R2 = 0.54717767


#### 3. 모델3

In [74]:
model = sm.OLS.from_formula('로그연봉 ~ C(시즌) +C(팀) + C(나이C):scale(횟수) + C(포지션) + scale(G) + scale(홈런) + scale(루타) +\
                            scale(도루) +scale(고4) + scale(타3) + C(외국인) + scale(BABIP) +C(이적C) + scale(ISOD) + \
                            scale(BBK) + scale(타율) + scale(경험)', data=train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     19.79
Date:                Thu, 02 Apr 2020   Prob (F-statistic):           4.50e-82
Time:                        21:17:30   Log-Likelihood:                -216.09
No. Observations:                 485   AIC:                             530.2
Df Residuals:                     436   BIC:                             735.2
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               9.9805    

In [75]:
df = train
from sklearn.model_selection import KFold

scores = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]
    
    model = sm.OLS.from_formula('로그연봉 ~ C(시즌) +C(팀) + C(나이C):scale(횟수) + C(포지션) + scale(G) + scale(홈런) + scale(루타) +\
                            scale(도루) +scale(고4) + scale(타3) + C(외국인) + scale(BABIP) +C(이적C) + scale(ISOD) + \
                            scale(BBK) + scale(타율) + scale(경험)' , data=df_train)
    result = model.fit()
    
    pred = result.predict(df_test)
    rss = ((df_test.로그연봉 - pred) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean())** 2).sum()
    rsquared = 1 - rss / tss
    
    scores[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result.rsquared, rsquared))

학습 R2 = 0.69761969, 검증 R2 = 0.61152107
학습 R2 = 0.70061104, 검증 R2 = 0.54803942
학습 R2 = 0.67896484, 검증 R2 = 0.66771108
학습 R2 = 0.69243782, 검증 R2 = 0.60517417
학습 R2 = 0.69336666, 검증 R2 = 0.60310650


#### 4. 모델 4

In [79]:
model = sm.OLS.from_formula('로그연봉 ~ C(시즌) + C(팀) + C(포지션) + G +\
                            홈런 + 루타 + 타석 + 득점 +\
                            도루 + 고4 + 타3 + 희타 + 희비 + 타율 + OPS +\
                            C(외국인) + C(나이C):횟수', data=train)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                   로그연봉   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.597
Method:                 Least Squares   F-statistic:                     16.56
Date:                Thu, 02 Apr 2020   Prob (F-statistic):           6.02e-70
Time:                        21:38:58   Log-Likelihood:                -252.23
No. Observations:                 485   AIC:                             598.5
Df Residuals:                     438   BIC:                             795.1
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         9.4750      0.253     37.457

In [80]:
df = train
from sklearn.model_selection import KFold

scores = np.zeros(5)
cv = KFold(5, shuffle=True, random_state=0)
for i, (idx_train, idx_test) in enumerate(cv.split(df)):
    df_train = df.iloc[idx_train]
    df_test = df.iloc[idx_test]
    
    model = sm.OLS.from_formula('로그연봉 ~ C(시즌) + C(팀) + C(포지션) + G +\
                            홈런 + 루타 + 타석 + 득점 +\
                            도루 + 고4 + 타3 + 희타 + 희비 + 타율 + OPS +\
                            C(외국인) + C(나이C):횟수' , data=df_train)
    result = model.fit()
    
    pred = result.predict(df_test)
    rss = ((df_test.로그연봉 - pred) ** 2).sum()
    tss = ((df_test.로그연봉 - df_test.로그연봉.mean())** 2).sum()
    rsquared = 1 - rss / tss
    
    scores[i] = rsquared
    print("학습 R2 = {:.8f}, 검증 R2 = {:.8f}".format(result.rsquared, rsquared))

학습 R2 = 0.65085263, 검증 R2 = 0.49514884
학습 R2 = 0.65990360, 검증 R2 = 0.40265988
학습 R2 = 0.63610197, 검증 R2 = 0.55490774
학습 R2 = 0.64067558, 검증 R2 = 0.53617170
학습 R2 = 0.64395462, 검증 R2 = 0.53501300
